# ===========================================
# 1_data_exploration.ipynb
# Analyse der FAQ-Daten und Wissensbasis im AskMeNow-Projekt
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from collections import Counter

# Für Inline-Plots (falls nicht automatisch aktiviert)
%matplotlib inline

# Stelle sicher, dass NLTK-Tokendaten vorhanden sind (nur beim ersten Mal)
nltk.download('punkt')

# ===========================================
# Zelle 2: Daten einlesen
# ===========================================
# Annahme: Die FAQ-Daten sind in einer CSV-Datei gespeichert, z.B. "faq_data.csv"
# Diese Datei enthält mindestens die Spalten "question" und "answer".
DATA_DIR = os.path.join("..", "data", "raw")
data_file = os.path.join(DATA_DIR, "faq_data.csv")

df = pd.read_csv(data_file)
print("Anzahl der FAQ-Einträge:", len(df))
display(df.head())

# ===========================================
# Zelle 3: Datenbereinigung (optional)
# ===========================================
def clean_text(text):
    """Entfernt HTML-Tags, überflüssige Leerzeichen und konvertiert den Text in Kleinbuchstaben."""
    if not isinstance(text, str):
        text = str(text)
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

# Bereinige Fragen und Antworten
df["clean_question"] = df["question"].apply(clean_text)
df["clean_answer"] = df["answer"].apply(clean_text)

print("\nBereinigte Beispiele:")
display(df[["clean_question", "clean_answer"]].head())

# ===========================================
# Zelle 4: Textlängen analysieren
# ===========================================
# Berechne die Länge der Fragen und Antworten (in Zeichen und in Tokens)
df["question_length"] = df["clean_question"].apply(len)
df["answer_length"] = df["clean_answer"].apply(len)
df["question_token_count"] = df["clean_question"].apply(lambda x: len(nltk.word_tokenize(x)))
df["answer_token_count"] = df["clean_answer"].apply(lambda x: len(nltk.word_tokenize(x)))

print("\nStatistiken zur Länge der Fragen (Zeichen):")
display(df["question_length"].describe())

print("\nStatistiken zur Länge der Antworten (Zeichen):")
display(df["answer_length"].describe())

# ===========================================
# Zelle 5: Visualisierung der Textlängen
# ===========================================
plt.figure(figsize=(10, 4))
plt.hist(df["question_token_count"], bins=30, color="steelblue", alpha=0.8)
plt.xlabel("Anzahl Tokens (Fragen)")
plt.ylabel("Anzahl Einträge")
plt.title("Verteilung der Token-Anzahl in den Fragen")
plt.show()

plt.figure(figsize=(10, 4))
plt.hist(df["answer_token_count"], bins=30, color="seagreen", alpha=0.8)
plt.xlabel("Anzahl Tokens (Antworten)")
plt.ylabel("Anzahl Einträge")
plt.title("Verteilung der Token-Anzahl in den Antworten")
plt.show()

# ===========================================
# Zelle 6: Häufigkeitsanalyse der Wörter
# ===========================================
# Tokenisierung der Fragen
df["question_tokens"] = df["clean_question"].apply(nltk.word_tokenize)
# Tokenisierung der Antworten
df["answer_tokens"] = df["clean_answer"].apply(nltk.word_tokenize)

# Erstelle einen Counter für alle Tokens in Fragen und Antworten
all_question_tokens = [token for tokens in df["question_tokens"] for token in tokens]
all_answer_tokens = [token for tokens in df["answer_tokens"] for token in tokens]

question_counter = Counter(all_question_tokens)
answer_counter = Counter(all_answer_tokens)

print("\nTop 10 häufigste Wörter in Fragen:")
print(question_counter.most_common(10))

print("\nTop 10 häufigste Wörter in Antworten:")
print(answer_counter.most_common(10))

plt.figure(figsize=(8, 4))
q_tokens, q_freqs = zip(*question_counter.most_common(10))
plt.bar(q_tokens, q_freqs, color="mediumpurple")
plt.xlabel("Wörter")
plt.ylabel("Häufigkeit")
plt.title("Top 10 Wörter in Fragen")
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8, 4))
a_tokens, a_freqs = zip(*answer_counter.most_common(10))
plt.bar(a_tokens, a_freqs, color="coral")
plt.xlabel("Wörter")
plt.ylabel("Häufigkeit")
plt.title("Top 10 Wörter in Antworten")
plt.xticks(rotation=45)
plt.show()

# ===========================================
# Zelle 7: Fazit & Nächste Schritte
# ===========================================
print("""
Fazit:
- Die FAQ-Daten umfassen eine Vielzahl von Fragen und Antworten mit unterschiedlichen Längen und Token-Anzahlen.
- Die Bereinigung und Tokenisierung liefern saubere, konsistente Texte.
- Die Häufigkeitsanalyse zeigt, welche Wörter in den Fragen und Antworten dominieren.

Nächste Schritte:
- Weitere Datenbereinigung und -vorbereitung in notebooks/2_data_preprocessing.ipynb.
- Aufteilung in Trainings-, Validierungs- und Test-Datensätze.
- Entwicklung und Training eines Q&A-Modells (z. B. Seq2Seq oder Transformer-basierte Ansätze) in notebooks/3_training_demo.ipynb.
""")
